<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Lora_test_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sd-scripts github: https://github.com/kohya-ss/sd-scripts

Reference article：
· https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/#i-7

· https://www.ipentec.com/document/software-stable-diffusion-lora-learn#section_12



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.10.12


In [2]:
!git clone https://github.com/kohya-ss/sd-scripts

Cloning into 'sd-scripts'...
remote: Enumerating objects: 3061, done.
remote: Counting objects: 100% (1474/1474), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 3061 (delta 1364), reused 1360 (delta 1317), pack-reused 1587
Receiving objects: 100% (3061/3061), 3.43 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (2119/2119), done.


In [3]:
#According to the sd-scripts github documentation, we should install PyTorch before install requirements.txt 
#because the versions of it depends on your environment and requirements.txt does not contain requirements for PyTorch.

%cd sd-scripts
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install  --upgrade -r requirements.txt
#!pip install -U --pre triton
!pip install xformers==0.0.16rc425
%cd ../

/content/sd-scripts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu117 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 30.4 MB/s eta 0:00:00
/content


In [4]:
# Change some config (Some user reports [ValueError: fp16 mixed precision requires a GPU] is occurred in training )
!accelerate config default --mixed_precision fp16

2023-06-12 03:17:46.998922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 03:17:47.165651: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 03:17:47.204565: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 03:17:48.078157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [5]:
# Copy training images folder
training_data_path='/content/sd-scripts/training_data'

%cd /content/sd-scripts
%cp -r /content/drive/MyDrive/datasets/imgs/resized_canbright_imgs_1_partial '{training_data_path}'

/content/sd-scripts


In [6]:
# Generate captions and tags for each image.

!python finetune/make_captions.py --batch_size 8 '{training_data_path}'
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 '{training_data_path}'

2023-06-12 04:50:26.705984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 04:50:26.873973: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 04:50:26.914812: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 04:50:27.780304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [7]:
# Merges captions with metadata and outputs the result in meta_cap.json.
# Merges Data Description (DD) tags with metadata from the previous step and generates a new file meta_cap_dd.json.
# Cleans and preprocesses the captions and tags from the previous step and generates a cleaned version in meta_clean.json.

!python finetune/merge_captions_to_metadata.py '{training_data_path}' meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py '{training_data_path}' --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

2023-06-12 04:51:29.985631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 04:51:30.153104: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 04:51:30.193855: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 04:51:31.043641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [8]:
!python finetune/prepare_buckets_latents.py \
'{training_data_path}' meta_clean.json meta_lat.json \
/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--batch_size 4 \
--max_resolution 512,512 \
--mixed_precision no

2023-06-12 04:51:43.864554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 04:51:44.031350: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 04:51:44.070918: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 04:51:44.921084: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [13]:
!pwd

/content/sd-scripts


In [14]:
!accelerate launch  \
--num_cpu_threads_per_process 1 \
train_network.py \
--pretrained_model_name_or_path=/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--in_json meta_lat.json \
--train_data_dir='{training_data_path}' \
--output_dir=output_models \
--shuffle_caption \
--train_batch_size=1 \
--learning_rate=1e-4 \
--max_train_steps=300 \
--use_8bit_adam \
--xformers --gradient_checkpointing \
--mixed_precision=fp16 \
--save_every_n_epochs=10 \
--save_precision=fp16 \
--save_model_as=safetensors  \
--network_module=networks.lora

2023-06-12 05:20:40.787371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 05:20:40.950364: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 05:20:40.989100: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 05:20:41.840101: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [15]:
model_path='./output_models/last.safetensors'
save_path='/content/drive/MyDrive/my_models/canbright_satoo/'
new_file_name = 'satoo_with_descriptions.safetensors'

!mkdir -p '{save_path}'
!cp '{model_path}' '{save_path}/{new_file_name}'

In [ ]:
# write dataset config
import toml
instance_data_dir='/content/drive/MyDrive/datasets/imgs/with_txt_resized_canbright_imgs_1_partial/'

dataset_config={
    'general': {
        'enable_bucket': True
    },
    'datasets': [
        {
            'resolution': 512,
            'batch_size': 4,
            'subsets': [
                {
                    'image_dir': instance_data_dir,
                    'caption_extension': '.txt',
                    'num_repeats': 1
                }
            ]
        }
    ]
}
dataset_config_path='/content/drive/MyDrive/ML_Practices/Lora_test_1/lora/dataset_config.toml'
with open(dataset_config_path, 'w') as file:
    toml.dump(dataset_config, file)